In [1]:
import sys
sys.path.append('../')
from utilities import *

# Load data

In [2]:
corpus_dir = '../../data/corpus/'
model_dir = '../../data/gpt2/'

In [3]:
noisy = pd.read_csv(f'{corpus_dir}noisy.csv')

In [4]:
noisy.shape

(50, 2)

# Load the GPT-2 Language Model

In [5]:
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt2.to(device)
_ = gpt2.eval()

100%|██████████| 548118077/548118077 [05:45<00:00, 1588622.22B/s] 


In [6]:
channel = NoisyChannelModel(gpt2)

100%|██████████| 456318/456318 [00:00<00:00, 1718299.43B/s]


## Poisson Channel model

In [7]:
def best_l(l):
    channel.l = l
    df = channel.beam_search_sentences(noisy.err)
    return np.array([wer(correct, changed) for correct, changed in zip(noisy.cor, df[0].str.lower())]).mean()

In [8]:
channel.channel_method_poisson = True
channel.channel_prob_param = 0.01

In [ ]:
best = fmin(fn=best_l, space=hp.uniform('l', 0.2, 5), algo=tpe.suggest, max_evals=20)

 25%|██▌       | 5/20 [33:00<1:39:07, 396.51s/it, best loss: 0.24178770992122758]

In [12]:
best['l']

1.1308563541963113

In [ ]:
with open(f'{model_dir}best.pickle', 'wb') as file:
    pickle.dump(best, file)